# Simple Online and Realtime Tracking

In [1]:
import numpy as np
from sort.sort import *

In [2]:
cd

/Users/flavienvidal


In [3]:
ls

Applications/                   MAPINF631/
Desktop/                        Movies/
Documents/                      Music/
Downloads/                      Pictures/
HadrienFlavienTradingProject/   Public/
INF581/                         PycharmProjects/
INF632_NLP/                     Untitled.ipynb
INF633/                         data/
INF634/                         deep_sort/
Interviews/                     flavien.zip
Library/                        gensim-data/
Lydia_techtest/                 nltk_data/
MAP555_Assignment1-Copy1.ipynb  opt/
MAP555_Assignment2_FV.ipynb     part5/
MAP569/                         sort/
MAPIN/                          text.txt
MAPINF630/


In [4]:
all_outputs_path = "/Users/flavienvidal/INF634/all_outputs"
labels_path = "/Users/flavienvidal/INF634/all_outputs/labels"

all_outputs = list(os.listdir("/Users/flavienvidal/INF634/all_outputs/labels"))
all_outputs.sort()

In [5]:
all_outputs[:10]

['002400.txt',
 '002401.txt',
 '002402.txt',
 '002403.txt',
 '002404.txt',
 '002405.txt',
 '002406.txt',
 '002407.txt',
 '002408.txt',
 '002409.txt']

In [6]:
# get detections
path_to_detections = "/Users/flavienvidal/INF634/all_outputs/labels"
path_to_detection_002400 = "/Users/flavienvidal/INF634/all_outputs/labels/002400.txt"

detections = np.loadtxt(path_to_detection_002400)
detections

array([[ 534.,  196.,   63.,   77.],
       [ 271.,  205.,   44.,   79.],
       [ 999.,  308.,   42.,   43.],
       [1036.,  208.,   54.,   74.]])

In [7]:
def remove_useless_files(path: str='/Users/flavienvidal/INF634'):
    all_files = list(os.listdir(path))
    all_files.sort()
    all_files
    for x in all_files:
        if 'frame' and '.jpg' in x:
            os.remove(x)

remove_useless_files()

In [8]:
studied_frames_res = list(os.listdir('/Users/flavienvidal/INF634'))
studied_frames_res.sort()
studied_frames_res

['.DS_Store',
 '.idea',
 '.ipynb_checkpoints',
 'SORT_tracker.ipynb',
 'SORT_tracker_new.ipynb',
 'all_outputs',
 'data',
 'episode01',
 'flavien_det.txt',
 'output',
 'output.avi',
 'part15-30.avi',
 'part50-79.avi',
 'sort',
 'venv',
 'yolov4-deepsort']

In [9]:
def create_subsets_of_episodes(starting_frame: int, ending_frame: int):
    starting_frame = '%06d.jpg'%(starting_frame)
    ending_frame = '%06d.jpg'%(ending_frame)
    return starting_frame, ending_frame

create_subsets_of_episodes(2400, 2650)

('002400.jpg', '002650.jpg')

In [10]:
cd INF634

/Users/flavienvidal/INF634


In [11]:
def change_first_format(path_to_final_res: str = "./data/train/which_frames/det/det.txt", 
                        path_to_current_det: str = "./all_outputs/labels"):
    """
    correct "det.txt" format is as follow with a signle comma and no space between the figures: 
    frame_id,-1,xmin,ymin,w,h,confidence,-1,-1,-1
    """

    confidence = 0 # todo: set to 0 for now but need to change it later with Ayham's predictions confidences

    studied_frames_res = list(os.listdir(path_to_current_det))
    studied_frames_res.sort() # typically: studied_frames_res = ['002400.txt', '002401.txt', '002402.txt', ...]

    starting_frame = int(studied_frames_res[0].replace(".txt", ""))
    ending_frame = int(studied_frames_res[-1].replace(".txt", ""))

    path_to_final_res = path_to_final_res.replace("which_frames", f"frames_{starting_frame}_to_{ending_frame}")

    if not os.path.exists(path_to_final_res.replace("det.txt", "")):
        os.makedirs(path_to_final_res.replace("det.txt", ""))
        
    final_det = open(path_to_final_res, "w")
    
    all_files =[]
    for frame in studied_frames_res:
        all_files.append(os.path.join(path_to_current_det, frame))
        #print(type(files))

    for frame in all_files:  # todo: [:4] to remove
        file = open(frame, "r") 
        for line in file.readlines():
            #print(f"\u25A0 frame = {frame}")
            #print(f"  file = {file}") 
            #print(f"  line = {line}") 

            L = line.split(" ")
            #print(f"  L = {L}")

            frame_id = int(frame.split("/")[-1].split(".txt")[0])
            xmin = L[0]
            ymin = L[1]
            w = L[2]
            h = L[3].replace("\n", "")

            final_det.write('{},{},{},{},{},{},{},{},{},{}\n'.format(frame_id, -1, xmin, ymin, w, h, confidence, -1, -1, -1)) 
    final_det.close()


    
    
    

In [12]:
change_first_format(path_to_final_res = "./data/train/which_frames/det/det.txt", path_to_current_det = "./all_outputs/labels")

In [13]:
import numpy as np
from sort.sort import *

In [14]:
import argparse

parser = argparse.ArgumentParser(description='SORT demo')
parser.add_argument('-f') # adding a dummy parser argument in the code worked: https://stackoverflow.com/questions/42249982/systemexit-2-error-when-calling-parse-args-within-ipython
parser.add_argument('--display', dest='display', help='Display online tracker output (slow) [False]',action='store_true')
parser.add_argument("--seq_path", help="Path to detections.", type=str, default='data')
parser.add_argument("--phase", help="Subdirectory in seq_path.", type=str, default='train')
parser.add_argument("--max_age", 
                    help="Maximum number of frames to keep alive a track without associated detections.", 
                    type=int, default=1)
parser.add_argument("--min_hits", 
                    help="Minimum number of associated detections before track is initialised.", 
                    type=int, default=3)
parser.add_argument("--iou_threshold", help="Minimum IOU for match.", type=float, default=0.3)
args = parser.parse_args()




In [15]:
args = parser.parse_args("--display --seq_path data --phase train --max_age 1 --min_hits 3 --iou_threshold 0.3".split())

In [16]:
#args = parser.parse_args("--seq_path data --phase train --max_age 1 --min_hits 3 --iou_threshold 0.3".split())

In [17]:
args

Namespace(display=True, f=None, iou_threshold=0.3, max_age=1, min_hits=3, phase='train', seq_path='data')

Total Tracking took: 0.303 seconds for 251 frames or 828.4 FPS
Note: to get real runtime results run without the option: --display
        
Total Tracking took: 0.314 seconds for 251 frames or 799.1 FPS
Note: to get real runtime results run without the option: --display

Total Tracking took: 0.317 seconds for 251 frames or 790.7 FPS
0.3174474239349365
Note: to get real runtime results run without the option: --display

Total Tracking took: 0.315 seconds for 251 frames or 797.5 FPS
0.31471776962280273
Note: to get real runtime results run without the option: --display

Total Tracking took: 0.320 seconds for 251 frames or 785.2 FPS
0.31966733932495117
Note: to get real runtime results run without the option: --display


Total Tracking took: 0.624 seconds for 251 frames or 402.0 FPS
0.6244254112243652

Total Tracking took: 0.616 seconds for 251 frames or 407.6 FPS
0.6158390045166016

Total Tracking took: 0.653 seconds for 251 frames or 384.2 FPS
0.6532487869262695

Total Tracking took: 0.677 seconds for 251 frames or 371.0 FPS
0.6765975952148438

Total Tracking took: 0.623 seconds for 251 frames or 402.9 FPS
0.6229193210601807

In [8]:
import glob

# if __name__ == '__main__': # to incorporate in the final .py

display = args.display
phase = args.phase
colours = np.random.rand(32, 3)

if(display):
    plt.ion()
    fig = plt.figure()
    ax1 = fig.add_subplot()

if not os.path.exists('output'):
    os.makedirs('output')

# pattern is the path to all det.txt detection files
pattern = os.path.join(args.seq_path, phase, '*', 'det', 'det.txt')

print(f"glob.glob(pattern) = {print(glob.glob(pattern))}")
for seq_dets_fn in glob.glob(pattern): # takes all det.txt files one by one
    print(f"seq_dets_fn = {seq_dets_fn}")
    mot_tracker = Sort(max_age=args.max_age, 
                       min_hits=args.min_hits,
                       iou_threshold=args.iou_threshold) #create instance of the SORT tracker
    seq_dets = np.loadtxt(seq_dets_fn, delimiter=',')
    print(f"seq_dets = {seq_dets}")
    seq = seq_dets_fn[pattern.find('*'):].split(os.path.sep)[0]
    print(f"seq = {seq}")

    with open(os.path.join('output', '%s.txt'%(seq)),'w') as out_file:
        print("Processing %s."%(seq))
        for frame in range(int(seq_dets[:, 0].min())-1, int(seq_dets[:, 0].max())): # modofied vrs
            print(f"\n\u25A0 frame = {frame} (min = {int(seq_dets[:, 0].min())}, max = {int(seq_dets[:,0].max())})")
            frame += 1 #detection and frame numbers begin at 1
            print(f"  seq_dets[:, 0]==frame = {seq_dets[:, 0]==frame}")
            dets = seq_dets[seq_dets[:, 0]==frame, 2:7]
            # Will typically be of the form:
            # dets = [[ 534.  196.   63.   77.    0.]
            #         [ 271.  205.   44.   79.    0.]
            #         [ 999.  308.   42.   43.    0.]
            #         [1036.  208.   54.   74.    0.]]

            print(f"  dets = {dets}")
            dets[:, 2:4] += dets[:, 0:2] # convert initial [x1,y1,w,h] to [x1,y1,x2,y2]
            print(f"  dets = {dets}")

            if(display):
                #fn = os.path.join('mot_benchmark', phase, seq, 'img1', '%06d.jpg'%(frame))
                fn = os.path.join('episode01', '%06d.jpg'%(frame))
                print(f"    \u21FE fn = {fn}")
                im = io.imread(fn)
                ax1.imshow(im)
                plt.title(seq + ' Tracked Targets')

            start_time = time.time()
            trackers = mot_tracker.update(dets)
            print(f"  trackers = {trackers}")
            cycle_time = time.time() - start_time


            for d in trackers:
                print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]))
                print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]),file=out_file)
                if(display):
                    d = d.astype(np.int32)
                    ax1.add_patch(patches.Rectangle((d[0],d[1]),d[2]-d[0],d[3]-d[1],fill=False,lw=3,ec=colours[d[4]%32,:]))

            if(display):
                fig.canvas.flush_events()
                plt.draw()
                ax1.cla()

    if(display):
        print("Note: to get real runtime results run without the option: --display")




['data/train/frames_2400_to_2650/det/det.txt']
glob.glob(pattern) = None
seq_dets_fn = data/train/frames_2400_to_2650/det/det.txt
seq_dets = [[ 2.40e+03 -1.00e+00  5.34e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 [ 2.40e+03 -1.00e+00  2.71e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 [ 2.40e+03 -1.00e+00  9.99e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 ...
 [ 2.65e+03 -1.00e+00  9.12e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 [ 2.65e+03 -1.00e+00  2.25e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]
 [ 2.65e+03 -1.00e+00  1.38e+02 ... -1.00e+00 -1.00e+00 -1.00e+00]]
seq = frames_2400_to_2650
Processing frames_2400_to_2650.

■ frame = 2399 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [ True  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False Fal


■ frame = 2401 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False  True
  True  True  True  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2403 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False  True  True  True
  True False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2405 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True  True  True False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2407 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False  True  True
  True  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa

...


■ frame = 2641 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2643 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2645 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2647 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa


■ frame = 2649 (min = 2400, max = 2650)
  seq_dets[:, 0]==frame = [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False Fa